# Download file and explore file metadata

In [67]:
!wget https://s3.amazonaws.com/imcbucket/data/nasa.dat

--2022-01-24 14:30:54--  https://s3.amazonaws.com/imcbucket/data/nasa.dat
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.133.40
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.133.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205242368 (196M) [application/octet-stream]
Saving to: ‘nasa.dat.2’

nasa.dat.2          100%[===================>] 195.73M  45.0MB/s    in 4.6s    

2022-01-24 14:30:59 (42.7 MB/s) - ‘nasa.dat.2’ saved [205242368/205242368]



In [68]:
! ls -l nasa*

-rw-r--r-- 1 root root 205242368 Sep  6  2013 nasa.dat
-rw-r--r-- 1 root root 205242368 Sep  6  2013 nasa.dat.1
-rw-r--r-- 1 root root 205242368 Sep  6  2013 nasa.dat.2


In [69]:
! head -3 nasa.dat

199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245
unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985
199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085


In [70]:
! wc -l nasa.dat

1891714 nasa.dat


# Put the file into hdfs

In [71]:
! hdfs dfs -mkdir -p /rawzone/

In [72]:
! hdfs dfs -put nasa.dat /rawzone/

put: `/rawzone/nasa.dat': File exists


In [73]:
! hdfs dfs -ls /rawzone/nasa.dat

-rw-r--r--   2 root hadoop  205242368 2022-01-24 11:52 /rawzone/nasa.dat


# Data Pre-processing

## Read text file to dataframe

In [93]:
import pyspark.sql.functions as F

In [94]:
raw_df = spark.read.text('/rawzone/nasa.dat')

In [95]:
raw_df.printSchema()

root
 |-- value: string (nullable = true)



In [96]:
raw_df.show(5)

+--------------------+
|               value|
+--------------------+
|199.72.81.55 - - ...|
|unicomp6.unicomp....|
|199.120.110.21 - ...|
|burger.letters.co...|
|199.120.110.21 - ...|
+--------------------+
only showing top 5 rows



## Pattern matching

In [97]:
APACHE_ACCESS_LOG_PATTERN = '(\S*) - - \[(\d{2})\/(\S*)\/(\d{4}):(\d{2}):(\d{2}):(\d{2}) (\S*)\]'
adapted_log_pattern = r'(\S*) - - \[((\d{2})\/(\S*)\/(\d{4}):(\d{2}):(\d{2}):(\d{2}) (\S*))\]'

In [98]:
parsed_df = (
    raw_df
    .withColumn('ip_addr', F.regexp_extract('value', adapted_log_pattern, 1))
    .withColumn('date_str', F.regexp_extract('value', adapted_log_pattern, 2))
    .withColumn('timestamp_utc', F.to_timestamp(F.col('date_str'), 'dd/MMM/yyyy:HH:mm:ss Z'))
    .withColumn('timestamp_local', F.to_timestamp(F.col('date_str'), 'dd/MMM/yyyy:HH:mm:ss'))
)

In [99]:
parsed_df.printSchema()

root
 |-- value: string (nullable = true)
 |-- ip_addr: string (nullable = true)
 |-- date_str: string (nullable = true)
 |-- timestamp_utc: timestamp (nullable = true)
 |-- timestamp_local: timestamp (nullable = true)



In [100]:
parsed_df.show(5)

+--------------------+--------------------+--------------------+-------------------+-------------------+
|               value|             ip_addr|            date_str|      timestamp_utc|    timestamp_local|
+--------------------+--------------------+--------------------+-------------------+-------------------+
|199.72.81.55 - - ...|        199.72.81.55|01/Jul/1995:00:00...|1995-07-01 04:00:01|1995-07-01 00:00:01|
|unicomp6.unicomp....|unicomp6.unicomp.net|01/Jul/1995:00:00...|1995-07-01 04:00:06|1995-07-01 00:00:06|
|199.120.110.21 - ...|      199.120.110.21|01/Jul/1995:00:00...|1995-07-01 04:00:09|1995-07-01 00:00:09|
|burger.letters.co...|  burger.letters.com|01/Jul/1995:00:00...|1995-07-01 04:00:11|1995-07-01 00:00:11|
|199.120.110.21 - ...|      199.120.110.21|01/Jul/1995:00:00...|1995-07-01 04:00:11|1995-07-01 00:00:11|
+--------------------+--------------------+--------------------+-------------------+-------------------+
only showing top 5 rows



# Create final dataframe

In [101]:
final_df = parsed_df.select(
    'timestamp_utc', 'timestamp_local', 'ip_addr',
#     F.year('timestamp_utc').alias('year'), 
#     F.month('timestamp_utc').alias('month'), 
#     F.dayofmonth('timestamp_utc').alias('day_of_month'),
#     F.hour('timestamp_utc').alias('hour'),
#     F.minute('timestamp_utc').alias('minute'),
#     F.second('timestamp_utc').alias('second'),
)

In [102]:
final_df.printSchema()

root
 |-- timestamp_utc: timestamp (nullable = true)
 |-- timestamp_local: timestamp (nullable = true)
 |-- ip_addr: string (nullable = true)



In [103]:
final_df.show(5)

+-------------------+-------------------+--------------------+
|      timestamp_utc|    timestamp_local|             ip_addr|
+-------------------+-------------------+--------------------+
|1995-07-01 04:00:01|1995-07-01 00:00:01|        199.72.81.55|
|1995-07-01 04:00:06|1995-07-01 00:00:06|unicomp6.unicomp.net|
|1995-07-01 04:00:09|1995-07-01 00:00:09|      199.120.110.21|
|1995-07-01 04:00:11|1995-07-01 00:00:11|  burger.letters.com|
|1995-07-01 04:00:11|1995-07-01 00:00:11|      199.120.110.21|
+-------------------+-------------------+--------------------+
only showing top 5 rows



# Write to Hive

In [104]:
final_df

DataFrame[timestamp_utc: timestamp, timestamp_local: timestamp, ip_addr: string]

In [105]:
final_df.write.mode('overwrite').saveAsTable('nasa_webaccesslog')

In [106]:
! hive -e "describe nasa_webaccesslog"


Logging initialized using configuration in file:/etc/hive/conf.dist/hive-log4j2.properties Async: true
OK
timestamp_utc       	timestamp           	                    
timestamp_local     	timestamp           	                    
ip_addr             	string              	                    
Time taken: 1.444 seconds, Fetched: 3 row(s)


In [107]:
!hive -e "select * from nasa_webaccesslog where timestamp_local between '1995-07-01' and '1995-08-01' limit 10;"


Logging initialized using configuration in file:/etc/hive/conf.dist/hive-log4j2.properties Async: true
Query ID = root_20220124143924_8fdb92b2-a91b-4a65-9ed8-df0075547801
Total jobs = 1
Launching Job 1 out of 1
Status: Running (Executing on YARN cluster with App id application_1643024600486_0012)

OK
1995-07-01 04:00:01	1995-07-01 00:00:01	199.72.81.55
1995-07-01 04:00:06	1995-07-01 00:00:06	unicomp6.unicomp.net
1995-07-01 04:00:09	1995-07-01 00:00:09	199.120.110.21
1995-07-01 04:00:11	1995-07-01 00:00:11	burger.letters.com
1995-07-01 04:00:11	1995-07-01 00:00:11	199.120.110.21
1995-07-01 04:00:12	1995-07-01 00:00:12	burger.letters.com
1995-07-01 04:00:12	1995-07-01 00:00:12	burger.letters.com
1995-07-01 04:00:12	1995-07-01 00:00:12	205.212.115.106
1995-07-01 04:00:13	1995-07-01 00:00:13	d104.aa.net
1995-07-01 04:00:13	1995-07-01 00:00:13	129.94.144.152
Time taken: 18.249 seconds, Fetched: 10 row(s)
